In [128]:
import os
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_121065/65876671.py:31: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [129]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[4700994009, 9056663009, 6712674003, 466307400..."
1,Name,object,14,0.00,779583,"[CAPITAL CITY BLIMPIE, INC., SOUND PROPELLER S..."
2,City,object,30,0.00,32581,"[WOODRIDGE, RONKONKOMA, SAN DIEGO, GREENSBORO ..."
3,State,object,14,0.00,51,"[AZ, NY, MA, NJ]"
4,Zip,int64,0,0.00,33611,"[62221, 95624, 6051, 4401, 8302]"
5,Bank,object,1559,0.17,5802,"[BANK OF AMERICA NATL ASSOC, WELLS FARGO BANK ..."
6,BankState,object,1566,0.17,56,"[UT, OH, IL, CA, MO]"
7,NAICS,int64,0,0.00,1312,"[0, 311513]"
8,ApprovalDate,object,0,0.00,9859,"[20-Jan-06, 21-Feb-03, 5-Jun-98, 4-May-06, 3-N..."
9,ApprovalFY,object,0,0.00,70,"[1995, 2007, 1998, 2006, 1996]"


In [130]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [131]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [132]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [133]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [134]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [135]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [136]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [137]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [138]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [139]:
df.shape

(675026, 13)

In [140]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[54, 44-45, 81, 53, 51]"
1,Term,int64,0,0.0,377,"[84, 264, 73, 240]"
2,NoEmp,int64,0,0.0,499,"[3, 90, 5, 1]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 2]"
5,RetainedJob,int64,0,0.0,323,"[1, 0, 11, 2]"
6,UrbanRural,int64,0,0.0,3,"[1, 0]"
7,RevLineCr,object,0,0.0,2,[N]
8,LowDoc,object,0,0.0,2,"[Y, N]"
9,GrAppv,int64,0,0.0,16880,"[150000, 15000, 240000, 40000]"


In [141]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_0.csv


In [142]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_121065/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [143]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [144]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [145]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [146]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [147]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [148]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [149]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [150]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [151]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [152]:
df.shape

(460816, 13)

In [153]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[42, 54, 31-33, 56]"
1,Term,int64,0,0.0,373,"[48, 84, 240, 159]"
2,NoEmp,int64,0,0.0,432,"[5, 1, 4, 7, 15]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,192,"[10, 0]"
5,RetainedJob,int64,0,0.0,276,"[0, 2]"
6,UrbanRural,int64,0,0.0,3,"[0, 1, 2]"
7,RevLineCr,object,0,0.0,2,[N]
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,12824,"[938000, 75000, 50000, 65000]"


In [154]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_1.csv


In [155]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_121065/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [156]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [157]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [158]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [159]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [160]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [161]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [162]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [163]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [164]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [165]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [166]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [167]:
df.shape

(675026, 13)

In [168]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[72, 62, 51, 54]"
1,Term,int64,0,0.0,377,"[300, 60, 84, 240]"
2,NoEmp,int64,0,0.0,499,"[1, 80, 29, 2, 3]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 2, 1]"
5,RetainedJob,int64,0,0.0,323,"[5, 35, 1, 0, 3]"
6,UrbanRural,int64,0,0.0,3,"[1, 0, 2]"
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,16880,"[10000, 76000, 125000, 30000, 50000]"


In [169]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_2.csv


In [170]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_121065/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [171]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[4245834004, 2768074001, 1706144009, 566279300..."
1,Name,object,14,0.00,779583,"[VESTEX, CA. PARTNERSHIP, Garrett Shirley, PIE..."
2,City,object,30,0.00,32581,"[ZANESVILLE, Denver, IRVINE, CHARLESTOWN, HUNT..."
3,State,object,14,0.00,51,"[FL, WV, NY, TX]"
4,Zip,int64,0,0.00,33611,"[13501, 84032, 22304, 35226, 10011]"
5,Bank,object,1559,0.17,5802,"[FIFTH THIRD BANK, CITIZENS BANK NATL ASSOC, W..."
6,BankState,object,1566,0.17,56,"[NH, CA, TX, DE, OR]"
7,NAICS,int64,0,0.00,1312,"[0, 334411, 423210, 532230]"
8,ApprovalDate,object,0,0.00,9859,"[23-Oct-08, 23-Apr-02, 15-Nov-96, 25-Feb-00, 1..."
9,ApprovalFY,object,0,0.00,70,"[2003, 1998, 1999, 2007]"


In [172]:
df = df.drop_duplicates(subset='LoanNr_ChkDgt')

In [173]:
is_valide = {1.0, 2.0}
df['NewExist'] = df['NewExist'].isin(is_valide)

In [174]:
is_valide = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].isin(is_valide)

In [175]:
is_valide = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].isin(is_valide)

In [176]:
df = df.dropna(subset='NAICS')
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [177]:
mapping = {0: 'unknown', 1: 'urban', 2: 'rural'}
df['UrbanRural'] = df['UrbanRural'].map(mapping).astype(str)

In [178]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1).astype(bool)
df = df.drop('FranchiseCode', axis=1)

In [179]:
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], format='%d-%b-%y').dt.strftime('%d-%m-%y')

In [180]:
df['ApprovalDate'].unique()

array(['28-02-97', '07-02-06', '25-03-97', ..., '25-02-97', '26-02-97',
       '27-02-97'], dtype=object)

In [181]:
df['ApprovalFY'] = df['ApprovalFY'].astype(str).apply(lambda x: '20' + x[-2:] if int(x[-2:]) >= 0 and int(x[-2:]) <= 27 else '19' + x[-2:])
df['ApprovalFY'].unique()

array(['1997', '2006', '1998', '1999', '2000', '2001', '2003', '2004',
       '2005', '1984', '2007', '1987', '2008', '1988', '2009', '1989',
       '1990', '2010', '1991', '2011', '1992', '2002', '2012', '1993',
       '2013', '1994', '2014', '1995', '1996'], dtype=object)

In [182]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [183]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & (df['ChgOffPrinGr'] != 0.0)]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [184]:
samples_bad_lol = df[df['ChgOffDate'].notna()]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [185]:
df['MIS_Status'].value_counts()

MIS_Status
P I F     550334
CHGOFF    145335
Name: count, dtype: int64

In [186]:
df['Approve'] = df['MIS_Status'].apply(lambda x: 1 if x == 'P I F' else 0).astype(bool)
df.drop(['MIS_Status'], axis=1, inplace=True)

In [187]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df = df.dropna()

In [188]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,City,object,0,0.0,28860,"[PENSACOLA, DOWNEY, BEMIDJI, FOLSOM, PUEBLO WEST]"
1,State,object,0,0.0,51,"[WV, VA, KS, CA]"
2,Bank,object,0,0.0,5221,"[UMPQUA BANK, PNC BANK, NATIONAL ASSOCIATION, ..."
3,BankState,object,0,0.0,55,"[IL, MA, RI, KS, TN]"
4,NAICS,object,0,0.0,20,"[44-45, 31-33, 62, 54, 72]"
5,ApprovalDate,object,0,0.0,7367,"[21-02-03, 25-01-10, 23-08-05, 23-08-02, 11-07..."
6,ApprovalFY,object,0,0.0,29,"[2011, 1992, 2008, 2005]"
7,Term,int64,0,0.0,378,"[126, 240, 12, 84, 180]"
8,NoEmp,int64,0,0.0,502,"[7, 6, 12, 4]"
9,NewExist,bool,0,0.0,2,[True]


In [189]:
df.shape

(696583, 19)

In [190]:
df_to_csv(df, folder_path, 'SBA_Cleaned_Pure.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_Pure.csv
